PART 1

In [15]:
import pandas as pd
import requests
import numpy as np
!pip install lxml
from lxml import html


In [16]:
page=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
tree = html.fromstring(page.content)
postal_code = tree.xpath('//*[@id="mw-content-text"]/div/table[1]/tbody/tr/td[1]/text()')
borough = tree.xpath('//*[@id="mw-content-text"]/div/table[1]/tbody/tr/td[2]/text()')
neighborhood = tree.xpath('//*[@id="mw-content-text"]/div/table[1]/tbody/tr/td[3]/text()')
for i in range(len(postal_code)):
    postal_code[i] = postal_code[i].strip()
    borough[i] = borough[i].strip()
    neighborhood[i] = neighborhood[i].strip()
    if borough[i] == 'Not assigned':
        borough[i] = np.nan
    if neighborhood[i] == 'Not assigned':
        neighborhood[i] = borough[i]
df = pd.DataFrame({'postal_code': postal_code, 'Borough': borough, 'Neighborhood':neighborhood})


In [17]:
df.dropna(inplace=True)
df

,postal_code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [18]:
df.shape

(103, 3)

# PART 2

In [28]:

latlons = pd.read_csv('http://cocl.us/Geospatial_data')
new_df = pd.merge(df, latlons, how='left', left_on = 'postal_code', right_on = 'Postal Code')
new_df = new_df.drop(columns = 'Postal Code')
new_df

,postal_code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


# PART 3

In [56]:
#map them
import folium
map_all = folium.Map(location=[43.6532, -79.3832], zoom_start=11)
color = ['red', 'blue', 'green', 'purple', 'grey', 'black', 'orange', 'yellow', 'white', 'brown']
i = 0
dfs = [new_df[new_df['Borough']=='North York'], new_df[new_df['Borough']=='Downtown Toronto'], new_df[new_df['Borough']=='Etobicoke'], new_df[new_df['Borough']=='Scarborough'], new_df[new_df['Borough']=='East York'], new_df[new_df['Borough']=='York'], new_df[new_df['Borough']=='East Toronto'], new_df[new_df['Borough']=='West Toronto'], new_df[new_df['Borough']=='Central Toronto'], new_df[new_df['Borough']=='Mississauga']]
for x in df['Borough'].unique():
    for lat, lng, label in zip(dfs[i]['Latitude'], dfs[i]['Longitude'], dfs[i]['Neighborhood']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker([lat, lng],radius=5,popup=label,color=color[i],fill=True,fill_color='#3186cc',fill_opacity=0.7,parse_html=False).add_to(map_all)
    i += 1
map_all
#i just have literally no idea what im supposed to cluster with and i really dont understand how to get all the venues and stuff